In [3]:
import spacy
from spacy.lang.pt.examples import sentences
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

data = pd.read_csv('prus (2).csv')

nlp = spacy.load('pt_core_news_sm')

cln = []

# Carregar dicionário de abreviaturas/gírias/erros
def setup_abbr():
	file = open("dic_portuguese.txt", encoding='utf-8')
	abbr_dict = {}

	for line in file:
		w = line.split(";")
		abbr_dict[w[0]] = w[1].replace("\n", "")
	file.close()

	return abbr_dict

abbr_dict = setup_abbr()

for i in range(len(data)):  
	doc = nlp(data['comments'][i])
	lemma_sentence = []
    
#transformando a sentença em caixa baixa
	doc_lower = doc.text.lower()

#remoção de pontuações, números e caracteres especiais
	doc_punctuation = re.sub('[^a-zãàáâéêíõóôúç \n]', ' ', doc_lower)
    
#correção ortográfica
	doc_corrected = nlp(" ".join([abbr_dict.get(w, w) for w in doc_punctuation.split()]))
    
#lematização  
	for token in doc_corrected:
		if token.pos_ == 'VERB':
			lemma = token.lemma_
			lemma_sentence.append(lemma)
		else:
			lemma_sentence.append(token.text)
	lemmatized_sentence = ' '.join(lemma_sentence)
    
#remoção das stop words
	stop_words = set(stopwords.words('portuguese')+ \
	["{user}", "{url}", "<br/>", "myfitnesspal", "sigaa", "neste"]) 
	stop_words.remove("não")
	stop_words.remove("sem")
	word_tokens = word_tokenize(lemmatized_sentence)
	filtered_sentence = [w for w in word_tokens if not w in stop_words] 
	filtered_sentence = [] 
	for w in word_tokens: 
		if w not in stop_words: 
			filtered_sentence.append(w)
	result = ' '.join(filtered_sentence)
    
	cln.append(result)
    
ser = pd.DataFrame(data=np.array(cln), columns= ['CommentClean'])
print(ser)
#ser.to_csv('pruNormalized (2).csv', index=False)

                                         CommentClean
0                                    não pego menores
1               travar toda vez seguida mensagem erro
2   vou cadastrar não abrir sempre fica dizer não ...
3                             travar ver gráfico peso
4                             não ser possível entrar
..                                                ...
94  acontecer comigo tempo matrícula passar aparec...
95  não conseguir começar ontem hoje não conseguir...
96                 não conseguir matricular oncologia
97  excluir sim algum motivo sistema não permitir ...
98  tentar fazer trancamento internet não encontra...

[99 rows x 1 columns]
